Below is all packages which we used for this project

In [23]:
import pandas as pd
import numpy as np
from scipy import sparse
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from scipy.sparse import hstack
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
import csv
from collections import defaultdict
from sklearn.linear_model import LinearRegression
# nltk.download('stopwords')
# nltk.download('punkt')
# nltk.download('wordnet')

In [10]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Below is the code to read both train.csv file and test.csv file and stored in dataframe df1(trian.csv) and df2(test.csv)

In [11]:
#reading the corresponding csv file
df1 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/mie1624/train.csv')
df2 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/mie1624/test.csv')

Below is the code for us to apply data cleaning. We only use the reviewText and summary column as selected features. For train data's reviewText and summary column cleaning, we use the regular expression modle to remove all special character, change all characters to lower case and use nltk stopwords libraray to remove stop words. Then we applied tf-idf to extract 1500 features and make as a matrix.

In [12]:
# train data cleaning

df_train = df1.drop(columns = ['image', 'unixReviewTime','reviewHash'])
df_train = df_train.dropna()

# Defining a function to clean reviews
def dataCleaning(tweetList):
    
    # remove all special characters
    regex = re.compile('[^a-zA-Z0-9 ]')
    tweetList1 = [regex.sub('', tw) for tw in tweetList]
    
    # change all characters to lowercase
    tweetList1 = [tw.lower() for tw in tweetList1]
        
    # remove stop words using nltk stopwords library
    stop_words = set(stopwords.words('english'))
    for i in range(len(tweetList1)):
        example_sent = tweetList1[i]
        word_tokens = word_tokenize(example_sent)
        filtered_sentence = [w for w in word_tokens if not w in stop_words]
        filtered_sentence = []
        for w in word_tokens:
            if w not in stop_words:
                filtered_sentence.append(w)
        tweetList1[i] = ' '.join(filtered_sentence)
        
    wn = nltk.WordNetLemmatizer()
    for i in range(len(tweetList1)):
        w_list = [wn.lemmatize(word) for word in tweetList1[i].split()]
        tweetList1[i] = " ".join(w_list)
        
    return(tweetList1)

allRatings = []
userRatings = defaultdict(list)
itemRatings = defaultdict(list)

data = df1

allRatings = data['overall'].to_list()

for index_ in data.index:
    rating = data.loc[index_,'overall']
    userID = data.loc[index_,'reviewerID']
    userRatings[userID].append(rating)
    
    itemID = data.loc[index_,'itemID']
    itemRatings[itemID].append(rating)

globalAverage = sum(allRatings)/len(allRatings)

userAverage = {}
for u in userRatings:
    userAverage[u] = sum(userRatings[u]) / len(userRatings[u])
itemAverage = {}
for i in itemRatings:
    itemAverage[i] = sum(itemRatings[i]) / len(itemRatings[i])

user_avg_array_train = []
for l in df_train['reviewerID'].tolist():
    user_avg_array_train.append(userAverage[l])
item_avg_array_train = []
for l in df_train['itemID'].tolist():
    item_avg_array_train.append(itemAverage[l])
     
summary_train = dataCleaning(df_train['summary'].tolist())
df_train['summary'] = summary_train
review_train = dataCleaning(df_train['reviewText'].tolist())
df_train['reviewText'] = review_train

# One-hot encode category columns
category_array_train = pd.get_dummies(df_train[['category']], prefix_sep='_', drop_first=True).values

text_train = df_train["reviewText"] + ' ' + df_train["summary"]

#Using the Tfidf Vectorizer on the summary and review
vectorizer = TfidfVectorizer(max_features=2000)
fit_vectorizer = vectorizer.fit(text_train.tolist())
tfidf_train = fit_vectorizer.transform(text_train.tolist())

Below is the code for us to apply data cleaning. We only use the reviewText and summary column as selected features. For test data's reviewText and summary column cleaning, we use the regular expression modle to remove all special character, change all characters to lower case and use nltk stopwords libraray to remove stop words. Then we applied tf-idf to extract 1500 features and make as a matrix.

In [13]:
# test data cleaning

df_test = df2.drop(columns = ['image', 'unixReviewTime','reviewHash'])

for column in df_test.columns:
    df_test[column].fillna(df_test[column].mode()[0], inplace=True)
    
user_avg_array_test = []
for l in df_test['reviewerID'].tolist():
    user_avg_array_test.append(userAverage[l])
item_avg_array_test = []
for l in df_test['itemID'].tolist():
    if l in itemAverage:
        item_avg_array_test.append(itemAverage[l])
    else:
        item_avg_array_test.append(globalAverage)

summary_test = dataCleaning(df_test['summary'].tolist())
df_test['summary'] = summary_test
review_test = dataCleaning(df_test['reviewText'].tolist())
df_test['reviewText'] = review_test

# One-hot encode category columns
category_array_test = pd.get_dummies(df_test[['category']], prefix_sep='_', drop_first=True).values

text_test = df_test["reviewText"] + ' ' + df_test["summary"]

tfidf_test = fit_vectorizer.transform(text_test.tolist())

Below we convert the tf-idf result based on both training dataset and testing dataset to array. Then asssigned the corresponding array to X_train and X_test. The y_train dataset will be the 'overall' column from the training dataset.

In [14]:
col1_train = tfidf_train
col2_train = np.asarray(user_avg_array_train).reshape(-1,1)
col3_train = np.asarray(item_avg_array_train).reshape(-1,1)
col4_train = category_array_train
# clean_matrix_train = hstack((col1_train, col2_train, col3_train, col4_train))
clean_matrix_train  = col1_train.toarray()

col1_test = tfidf_test
col2_test = np.asarray(user_avg_array_test).reshape(-1,1)
col3_test = np.asarray(item_avg_array_test).reshape(-1,1)
col4_test = category_array_test
# clean_matrix_test = hstack((col1_test, col2_test, col3_test, col4_test))
clean_matrix_test = col1_test.toarray()

# scaler = StandardScaler()
# X_train = scaler.fit_transform(clean_matrix_train.todense())
# X_test = scaler.fit_transform(clean_matrix_test.todense())
X_train = clean_matrix_train
X_test = clean_matrix_test
y_train = df_train['overall'].astype('int64')

We use linear regression for the first model to get trained. We first use the cross_val_score to find the mse for this model which is showing on the next cell. 

In [15]:
print('linear regression:')
for param in [1]:
    model = LinearRegression()
    scores = cross_val_score(model, X_train, y_train, scoring='neg_mean_squared_error', cv=5)
    mean_mse = scores.mean()
    print("mse:", mean_mse*-1)

linear regression:
mse: 0.5725764961062434


We used multinomial logistic regression from Sklearn for our second model. We performed hyperparameter tuning of the C parameter with 2-fold cross validation. The best logistic regression model, using a C value of 1, gives a training MSE value of 0.7372.

In [19]:
print('logistic regression:')
for param in [0.1,1]:
    model = LogisticRegression(C=param)
    scores = cross_val_score(model, X_train, y_train, scoring='neg_mean_squared_error', cv=2)
    mean_mse = scores.mean()
    print('C =', param, ", mse:", mean_mse*-1)

logistic regression:
C = 0.1 , mse: 0.9843080036113085
C = 1 , mse: 0.7372239937031677


We used multinomial Naive-Bayes from Sklearn for our third model. We performed hyperparameter tuning of the alpha parameter with 5-fold cross validation. The best Naive-Bayes model, using an alpha value of 0.0001, gives a training MSE value of 1.1891.

In [20]:
print('multinomial Naive Bayes:')
for param in [0.0001,0.001,0.01,0.1,1]:
    model = MultinomialNB(alpha=param)
    scores = cross_val_score(model, X_train, y_train, scoring='neg_mean_squared_error', cv=5)
    mean_mse = scores.mean()
    print('alpha =', param, ", mse:", mean_mse*-1)

multinomial Naive Bayes:
alpha = 0.0001 , mse: 1.1890776292309737
alpha = 0.001 , mse: 1.1890776292309737
alpha = 0.01 , mse: 1.1892910352557746
alpha = 0.1 , mse: 1.1906981769718807
alpha = 1 , mse: 1.2022354311952843


We used a 3-layered neural network from Keras for our fourth and last model. We used Rectified Linear Unit as the activation function for the hidden layer of model, and for the activation function of the output layer we used the Softmax function. We set our loss function to be categorical cross-entropy. For the model training process we used a batch size of 256 and 50 epochs. The results can be seen below>

In [24]:
encoder_nn = LabelEncoder()
encoder_nn.fit(y_train)
encoded_Y = encoder_nn.transform(y_train)
y_train_nn = np_utils.to_categorical(encoded_Y)

In [25]:
model = Sequential()
model.add(Dense(8, input_dim=2000, activation='relu'))
model.add(Dense(5, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['mse','accuracy'])
model.fit(X_train, y_train_nn, epochs=50, batch_size=256)

Epoch 1/50
586/586 [==============================] - 3s 4ms/step - loss: 1.2133 - mse: 0.1231 - accuracy: 0.6327
Epoch 2/50
586/586 [==============================] - 2s 4ms/step - loss: 0.7434 - mse: 0.0770 - accuracy: 0.7119
Epoch 3/50
586/586 [==============================] - 2s 4ms/step - loss: 0.6755 - mse: 0.0718 - accuracy: 0.7324
Epoch 4/50
586/586 [==============================] - 2s 4ms/step - loss: 0.6602 - mse: 0.0707 - accuracy: 0.7386
Epoch 5/50
586/586 [==============================] - 2s 4ms/step - loss: 0.6485 - mse: 0.0698 - accuracy: 0.7427
Epoch 6/50
586/586 [==============================] - 2s 4ms/step - loss: 0.6481 - mse: 0.0697 - accuracy: 0.7437
Epoch 7/50
586/586 [==============================] - 2s 4ms/step - loss: 0.6463 - mse: 0.0696 - accuracy: 0.7438
Epoch 8/50
586/586 [==============================] - 2s 4ms/step - loss: 0.6393 - mse: 0.0690 - accuracy: 0.7460
Epoch 9/50
586/586 [==============================] - 2s 4ms/step - loss: 0.6428 - mse: 

Since the lowest MSE value we got was from the linear regression model (around 0.58367), we will apply this model to the testing dataset to find the prediction. We name the prediction array as the lr_predictions and stored predictions in this list.

In [16]:
lr_best_model = LinearRegression()
lr_best_model_fit = lr_best_model.fit(X_train, y_train)
lr_predictions = lr_best_model_fit.predict(X_test)

In [17]:
lr_predictions

array([4.229565  , 4.68056887, 4.54808571, ..., 4.52654549, 4.58256319,
       3.8346842 ])

Below is the code that we implemented to write the linear regression predictions to a pre-defined csv file named as test_result.csv. Then we can manually copy the predictions to the rating_pairs.csv file that we are going to submit on the kaggle. 

In [ ]:
# outfile = open('test_result.csv','w')
# out = csv.writer(outfile)
# out.writerows(map(lambda x: [x], lr_predictions))
# outfile.close()